In [ ]:
from textblob import TextBlob
import re 


def analyze_sentiment(text):
	blob = TextBlob(text)
	return blob.sentiment.polarity


def analyze_subjectivity(text):
	blob = TextBlob(text)
	return blob.sentiment.subjectivity


def read_text_file(filename):
	return clear_text(open(filename).read())
	

def clear_text(text):
	return re.sub(r'[^\x00-\x7F]+',' ', text)




if __name__=="__main__":
	print (analyze_sentiment(augur))


In [ ]:
# -*- coding: utf-8 -*-


import requests
import re
from numpy import mean
from xmljson import badgerfish as bf
from xml.etree.ElementTree import fromstring


COINGECKO_MAIN = 'https://www.coingecko.com/en'

AUGUR_SUBREDDIT = 'https://www.reddit.com/r/Augur/'


NEXT_PAGE_PATTERN = '\?count=\d+&amp;after=[^"]+'
SECOND_NEXT_PAGE_PATTERN = '\?amp=&amp;count=\d+&amp;after=[^"]+'


def scrap_text(url):
    headers = {
        'User-Agent': 'Chrome/75.0.3770.90 '}
    res = requests.get(url, headers=headers)
    res.raise_for_status()
    return res.text


def get_coin_fundementals():
    raw_text = scrap_text(COINGECKO_MAIN).replace("\n", "")
    rows = re.findall("<tr>.+?</tr>", raw_text)
    for row in rows:
        # if "monero" in row:
        try:
            name = re.findall(r"crypto-symbol text-muted'>(.+?)</span>", row)[0]
            hash_algo = re.findall(r"<small>(.+?)</small>", row)[0]
            print( "NAME:", name)
            print ("HASH ALGO:", hash_algo)
            print
        except IndexError:
            pass



def parse_xml(raw):
    return bf.data(fromstring(raw))


def get_titles(subreddit_url):
    raw_text = scrap_text(subreddit_url)
    titles = re.findall('<a class="title.+?>(.+?)<', raw_text)
    next_url_suffix = re.findall(NEXT_PAGE_PATTERN, raw_text)
    next_url_suffix2 = re.findall(SECOND_NEXT_PAGE_PATTERN, raw_text)
    if next_url_suffix or next_url_suffix2:
        next_url_suffix = next_url_suffix if next_url_suffix else  next_url_suffix2
        number = int(re.findall("count=(\d+)", next_url_suffix[0])[0])
        if number > 100:
            return titles
        next_url = subreddit_url.split("?")[0] + next_url_suffix[0]
        return titles + get_titles(next_url)

    return titles


def average_sentiment(text, ignore_zeros=True, negative_only=False):
    if negative_only:
        return mean([x for x in [analyze_sentiment(line) for line in text] if x < 0])
    if ignore_zeros:
        return mean([x for x in [analyze_sentiment(line) for line in text] if x != 0])
    return mean([analyze_sentiment(line) for line in text])


def average_subjectivity(text, ignore_zeros=True, negative_only=False):
    if negative_only:
        return mean([x for x in [analyze_subjectivity(line) for line in text] if x < 0])
    if ignore_zeros:
        return mean([x for x in [analyze_subjectivity(line) for line in text] if x != 0])
    return mean([analyze_subjectivity(line) for line in text])


def full_analysis(currency_name, subreddit_url):
    titles = get_titles(subreddit_url)
    sentiment = average_sentiment(titles)
    subjectivity = average_subjectivity(titles)

    print (currency_name + ":")
    print ("Sentiment:", sentiment)
    print ("Subjectivity:", subjectivity)
    print




if __name__ == '__main__':
    get_coin_fundementals()
    quit()
    full_analysis("Augur", AUGUR_SUBREDDIT)